In [0]:
import pandas as pd
import datetime as dt
import numpy as np

import warnings
warnings.filterwarnings("ignore")
from datetime import datetime,timedelta
from dateutil import relativedelta

from sklearn.linear_model import LinearRegression,SGDRegressor,Ridge,Lasso

from azure.cosmosdb.table.tableservice import TableService
from azure.cosmosdb.table.models import Entity

import pickle

In [0]:
def get_data_from_table_storage_table(table_service, source_table, filter_query):
    """ Retrieve data from Table Storage """
    for record in table_service.query_entities(source_table, filter=filter_query):
        yield record
  
def set_table_service(storage_account_name: str, account_key: str):
    """ Set the Azure Table Storage service """
    return TableService(account_name=storage_account_name, account_key=account_key)

def get_data(table_service: str, source_table: str, filter_query: str):
    return get_dataframe_from_table_storage_table(table_service, source_table, filter_query)

def get_dataframe_from_table_storage_table(table_service: str, source_table: str, filter_query: str):
    """ Create a dataframe from table storage data """
    return spark.createDataFrame(get_data_from_table_storage_table(table_service, source_table, filter_query))

def loading_dcs_data(last_n_hours, columns_of_interest, rename_dict):
    pgl_storage_account_name = 'pgldatalake'
    pgl_account_key = 'xAra5DFz1dX4t8b131wxxJ8pTCntBOjMFUrxYIg60nwkFhCohFERqJbkfoMNnLyqAg6IzSe2IQeNZSV3Z+c88g=='

    shk_storage_account_name = 'shikharanalyticsdevstore'
    shk_account_key = '9xzNHW65P7AfzuouxFW4MQICKW6DAYskfIABi/YobTfFraH4ODihAxrMWryv/LcwV1PPOACDZ3jqRRu0J27dDg=='
    
    pgl_table_service = set_table_service(pgl_storage_account_name, pgl_account_key)
    last_ts = "2020-09-01T01:06:10.586867Z"
    filter_query = "Timestamp ge datetime'{}'".format(last_ts)
    df = get_data(pgl_table_service, 'DCS60TPD', filter_query).toPandas()
    df['DateTime'] = df['DateTime'] + timedelta(hours = 5, minutes = 30)
    df_v2 = df.set_index('DateTime')[columns_of_interest].rename(columns = rename_dict).sort_index()
    df_v2['Date'] = df_v2.index.date
#     df_v2
    return df_v2

In [0]:
pgl_storage_account_name = 'pgldatalake'
pgl_account_key = 'xAra5DFz1dX4t8b131wxxJ8pTCntBOjMFUrxYIg60nwkFhCohFERqJbkfoMNnLyqAg6IzSe2IQeNZSV3Z+c88g=='

shk_storage_account_name = 'shikharanalyticsdevstore'
shk_account_key = '9xzNHW65P7AfzuouxFW4MQICKW6DAYskfIABi/YobTfFraH4ODihAxrMWryv/LcwV1PPOACDZ3jqRRu0J27dDg=='

pgl_table_service = set_table_service(pgl_storage_account_name, pgl_account_key)
# last_ts = str(dt.datetime.now() - timedelta(hours=last_n_hours)).replace(' ','T') + 'Z'
last_ts = "2020-09-01T01:06:10.586867Z"
filter_query = "Timestamp ge datetime'{}'".format(last_ts)
df = get_data(pgl_table_service, 'DCS60TPD', filter_query).toPandas()
df['DateTime'] = df['DateTime'] + timedelta(hours = 5, minutes = 30)

In [0]:
mold_cooling_blowers_60.index.min(), mold_cooling_blowers_60.index.max()

Out[49]: (Timestamp('2020-09-17 13:39:18.467000'),
 Timestamp('2020-12-02 11:38:28.190000'))

In [0]:
mold_cooling_blowers_60 = df[['DateTime','ConveyorcoolingBlowerpressurecontroloutputvaluePTCCMVDB214DD82','ConveyorcoolingBlowerpressurePTCCPVDB214DD92',
   'ConveyorcoolingBlowerpressuresetvaluePTCCSVDB214DD20']]
mold_cooling_blowers_60.set_index(['DateTime'], inplace = True)
mold_cooling_blowers_60.resample('24H').mean().to_csv('/dbfs/mnt/datalake/exploratory/Furnace_Analytics/60TPD/mold_cooling_blowers_60')

,ConveyorcoolingBlowerpressurecontroloutputvaluePTCCMVDB214DD82,ConveyorcoolingBlowerpressurePTCCPVDB214DD92,ConveyorcoolingBlowerpressuresetvaluePTCCSVDB214DD20
count,77.000000,77.000000,77.0
mean,13.146440,538.626359,500.0
std,28.591119,30.106492,0.0
min,0.000000,454.218741,500.0
25%,0.000000,525.486102,500.0
50%,0.000000,545.986359,500.0
75%,1.746243,556.809083,500.0
max,99.986340,628.629294,500.0


In [0]:
[col for col in df.columns if "cc" in col.lower()]

Out[41]: ['ConveyorcoolingBlowerpressurePTCCPVDB214DD92',
 'ConveyorcoolingBlowerpressurecontroloutputvaluePTCCMVDB214DD82',
 'ConveyorcoolingBlowerpressuresetvaluePTCCSVDB214DD20',
 'ThermocoupleChanelLeftTCCHLPVDB249DD276',
 'ThermocoupleChanelRightTCCHRPVDB249DD350',
 'ThermocoupleChanelTCCHNLPVDB249DD1386',
 'ThermocoupleChimneyTCCHMNPVDB249DD1238']

In [0]:
df.head(3)

,BlockcoolingBlowerpressurePTBBPVDB212DD92,BlockcoolingBlowerpressurecontroloutputvaluePTBBMVDB212DD82,BlockcoolingBlowerpressuresetvaluePTBBSVDB212DD20,BurnercoolingAirpressurePTBCAPVDB244DD794,CHIMENYDRAFTDB240DD84,CURRENTELECTE1DB240DD60,CURRENTELECTE2DB240DD64,CURRENTELECTE3DB240DD68,CURRENTELECTE4DB240DD72,CURRENTELECTE5DB240DD76,CURRENTELECTE6DB240DD80,ChimmanyDraftpressureDPTCHMNPVDB244DD646,ConveyorcoolingBlowerpressurePTCCPVDB214DD92,ConveyorcoolingBlowerpressurecontroloutputvaluePTCCMVDB214DD82,ConveyorcoolingBlowerpressuresetvaluePTCCSVDB214DD20,DateTime,Distributorzone1controloutputvalueFOTDZ1MVDB250DD82,Distributorzone1temperatureFOTDZ1PVDB250DD92,Distributorzone1temperaturesetvalueFOTDZ1SVDB250DD20,Distributorzone2controloutputvalueFOTDZ2MVDB251DD82,Distributorzone2temperatureFOTDZ2PVDB251DD92,Distributorzone2temperaturesetvalueFOTDZ2SVDB251DD20,Distributorzone3controloutputvalueFOTDZ3MVDB252DD82,Distributorzone3temperatureFOTDZ3PVDB252DD92,Distributorzone3temperaturesetvalueFOTDZ3SVDB252DD20,Distributorzone4controloutputvalueFOTDZ4MVDB253DD82,Distributorzone4temperatureFOTDZ4PVDB253DD92,Distributorzone4temperaturesetvalueFOTDZ4SVDB253DD20,Distributorzone5controloutputvalueFOTDZ5MVDB254DD82,Distributorzone5temperatureFOTDZ5PVDB254DD92,Distributorzone5temperaturesetvalueFOTDZ5SVDB254DD20,E1E2KVADB240DD108,E3E4KVADB240DD112,E5E6KVADB240DD116,E9E10KVADB240DD124,EARTHELECTE1DB240DD36,EARTHELECTE2DB240DD40,EARTHELECTE3DB240DD44,EARTHELECTE4DB240DD48,EARTHELECTE5DB240DD52,...,ThermocoupleFH53RearCenterMiddleTC53RCMPVDB245DD202,ThermocoupleFH53RearCenterTopTC53RCTPVDB245DD128,ThermocoupleFH54EntryTC54EPVDB247DD54,ThermocoupleFH54FrontCenterBottomTC54FCBPVDB247DD942,ThermocoupleFH54FrontCenterMiddleTC54FCMPVDB247DD868,ThermocoupleFH54FrontCenterTopTC54FCTPVDB247DD794,ThermocoupleFH54FrontLeftBottomTC54FLBPVDB247DD720,ThermocoupleFH54FrontLeftMiddleTC54FLMPVDB247DD646,ThermocoupleFH54FrontLeftTopTC54FLTPVDB247DD572,ThermocoupleFH54FrontRightBottomTC54FRBPVDB247DD1164,ThermocoupleFH54FrontRightMiddleTC54FRMPVDB247DD1090,ThermocoupleFH54FrontRightTopTC54FRTPVDB247DD1016,ThermocoupleFH54MiddleCenterBottomTC54MCBPVDB247DD498,ThermocoupleFH54MiddleCenterMiddleTC54MCMPVDB247DD424,ThermocoupleFH54MiddleCenterTopTC54MCTPVDB247DD350,ThermocoupleFH54RearCenterBottomTC54RCBPVDB247DD276,ThermocoupleFH54RearCenterMiddleTC54RCMPVDB247DD202,ThermocoupleFH54RearCenterTopTC54RCTPVDB247DD128,ThermocoupleFluePassLeftInnerTCFPLIPVDB249DD942,ThermocoupleFluePassLeftOuterTCFPLOPVDB249DD1164,ThermocoupleFluePassRightInnerTCFPRIPVDB249DD1016,ThermocoupleFluePassRightOuterTCFPROPVDB249DD1090,ThermocoupleMelterBottom1TCMB1PVDB249DD424,ThermocoupleMelterBottom2TCMB2PVDB249DD498,ThermocoupleMelterBottom3TCMB3PVDB249DD572,ThermocoupleMelterBottom4TCMB4PVDB249DD646,ThermocoupleMelterBottom5TCMB5PVDB249DD720,ThermocoupleMelterCrown1TCMC1PVDB249DD54,ThermocoupleMelterCrown2TCMC2PVDB249DD128,ThermocoupleMelterCrown3TCMC3PVDB249DD202,ThermocoupleregenratoreCrownleftTCRCLPVDB249DD794,ThermocoupleregenratoreCrownrightTCRCRPVDB249DD868,ThroatcoolingBlowerpressurePTBTPVDB213DD92,ThroatcoolingBlowerpressurecontroloutputvaluePTBTMVDB213DD82,ThroatcoolingBlowerpressuresetvaluePTBTSVDB213DD20,Timestamp,VOLTELECTE1E2DB240DD24,VOLTELECTE3E4DB240DD28,VOLTELECTE5E6DB240DD32,etag
0,14.134838,100.0,15.0,2.740162,-395.100006,982.099976,990.200012,937.500000,928.799988,887.200012,885.400024,90.0,594.907410,0.0,500.0,2020-12-02 11:38:28.190,61.628021,1295.254639,1295.0,31.059431,1323.842529,1325.0,50.463551,1350.462891,1350.0,26.236246,1324.652832,1325.0,54.842373,1295.138916,1295.0,74.900002,67.800003,58.799999,0.0,41.599998,35.099998,38.200001,35.700001,30.6,...,3000.217041,3000.217041,3000.217041,3000.217041,3000.217041,3000.217041,3000.217041,3000.217041,3000.217041,3000.217041,3000.217041,3000.217041,3000.217041,3000.217041,3000.217041,3000.217041,3000.217041,3000.217041,584.842346,582.042114,343.324829,272.519714,1270.29187,1264.291504,1311.594849,1125.781494,1032.0

In [0]:
dcs_df = (df[['DateTime','SecondaryAirFlowvalueFTSAFPVDB205DD92', 'MelterGasflowsetvalueFTMGFSVDB202DD20','ThermocoupleMelterCrown3TCMC3PVDB249DD202']]
 .rename(columns = {'SecondaryAirFlowvalueFTSAFPVDB205DD92':'Air flow',
                   'MelterGasflowsetvalueFTMGFSVDB202DD20':'Fuel flow',
                   'ThermocoupleMelterCrown3TCMC3PVDB249DD202':'MCT 3'}))
dcs_df.head(3)

,DateTime,Air flow,Fuel flow,MCT 3
0,2020-12-02 11:38:28.190,4563.834961,481.0,1551.512207
1,2020-12-02 11:37:28.190,4495.314941,481.0,1551.212280
2,2020-12-02 11:36:28.190,4563.688477,481.0,1551.012207


In [0]:
dcs_df['DateTime'] = pd.to_datetime(dcs_df['DateTime'])
dcs_df.head(3)

,DateTime,Air flow,Fuel flow,MCT 3
0,2020-12-02 11:38:28.190,4563.834961,481.0,1551.512207
1,2020-12-02 11:37:28.190,4495.314941,481.0,1551.212280
2,2020-12-02 11:36:28.190,4563.688477,481.0,1551.012207


In [0]:
# dcs_df.set_index(["DateTime"], inplace = True)
# dcs_df_30mins = dcs_df.resample('30T').mean()
dcs_df_30mins["Date"] = pd.to_datetime(dcs_df_30mins.index.date)
dcs_df_30mins.head(4)

,Air flow,Fuel flow,MCT 3,Date
DateTime,,,,
2020-09-17 13:30:00,4457.498993,480.000000,1554.712494,2020-09-17
2020-09-17 14:00:00,4456.950423,479.666667,1554.372473,2020-09-17
2020-09-17 14:30:00,4325.575163,464.333333,1556.075928,2020-09-17
2020-09-17 15:00:00,4268.142204,459.333333,1558.429423,2020-09-17


In [0]:
## Furnace draw
# draw = pd.read_excel("/dbfs/mnt/datalake/exploratory/Furnace_Analytics/60TPD/Glass Draw Kosamba June-Nov'20.xlsx", sheet_name="Sheet1")
# draw['Draw'] = draw['       Quantity']/1000
draw["Date"] = pd.to_datetime(draw["Date"])
draw.head(3)

,Pstng Date,Quantity,Date,Draw
0,01.07.2020,51726,2020-07-01,51.726
1,02.07.2020,51763,2020-07-02,51.763
2,03.07.2020,51832,2020-07-03,51.832


In [0]:
dcs_df_30mins_v2 = dcs_df_30mins.reset_index().merge(draw[['Date', 'Draw']], on = ['Date'], how = 'left')
dcs_df_30mins_v2.head(3)

,DateTime,Air flow,Fuel flow,MCT 3,Date,Draw
0,2020-09-17 13:30:00,4457.498993,480.000000,1554.712494,2020-09-17,55.189
1,2020-09-17 14:00:00,4456.950423,479.666667,1554.372473,2020-09-17,55.189
2,2020-09-17 14:30:00,4325.575163,464.333333,1556.075928,2020-09-17,55.189


In [0]:
## NCV
# gail_ncv = pd.read_excel('/dbfs/mnt/datalake/exploratory/Furnace_Analytics/60TPD/NG CV Kosamba_11Nov.xlsx', sheet_name = 'GAIL CV')
# gail_ncv['From'] = pd.to_datetime(gail_ncv['From'])
# gail_ncv.set_index(['From'], inplace = True)
# gail_ncv_v2 = gail_ncv.resample('30T').mean().reset_index()
gail_ncv_v2.rename(columns = {'From':'DateTime'}, inplace = True)
gail_ncv_v2.head(3)


,DateTime,NCV Value
0,2019-07-01 00:00:00,9142.055
1,2019-07-01 00:30:00,9128.255
2,2019-07-01 01:00:00,9127.430


In [0]:
dcs_df_30mins_v3 = dcs_df_30mins_v2.merge(gail_ncv_v2, on = ['DateTime'], how = 'inner')
dcs_df_30mins_v3['ATF'] = dcs_df_30mins_v3['Air flow']/ dcs_df_30mins_v3['Fuel flow']
dcs_df_30mins_v3.head(3)

,DateTime,Air flow,Fuel flow,MCT 3,Date,Draw,NCV Value,ATF
0,2020-09-17 13:30:00,4457.498993,480.000000,1554.712494,2020-09-17,55.189,9600.195,9.286456
1,2020-09-17 14:00:00,4456.950423,479.666667,1554.372473,2020-09-17,55.189,9606.295,9.291766
2,2020-09-17 14:30:00,4325.575163,464.333333,1556.075928,2020-09-17,55.189,9619.690,9.315668


In [0]:
dcs_df_30mins_v3.to_csv('/dbfs/mnt/datalake/exploratory/Furnace_Analytics/60TPD/dcs_df_30mins_v3.csv', index = False)

In [0]:
def mape(a,b, lr = False):
    if lr == False:
        return np.abs((a-b)/a).mean()
    else:
        b = [x[0] for x in b]
        return np.abs((a-b)/a).mean()

In [0]:
from sklearn.tree import DecisionTreeRegressor 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression,SGDRegressor,Ridge,Lasso
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.metrics import r2_score


cols = ["NCV Value","ATF","Draw","MCT 3"]
cols_v2 = ["Fuel flow","NCV Value","ATF","Draw","MCT 3"]
model_df_v4 = dcs_df_30mins_v3[cols_v2].loc[(dcs_df_30mins_v3['Fuel flow'] >= 350)].dropna()
train_v1 = model_df_v4.reset_index(drop = True).loc[:model_df_v4.shape[0]*0.8,]
X_train = train_v1[cols]
y_train = train_v1["Fuel flow"]
test_v1 = model_df_v4.dropna().reset_index(drop = True).loc[model_df_v4.shape[0]*0.8:,]
X_test = test_v1[cols]
y_test = test_v1["Fuel flow"]


dtree=DecisionTreeRegressor(max_depth = 3, random_state=101)
dtree.fit(X_train,y_train)

# Predictions
preds = dtree.predict(X_test)
preds_train = dtree.predict(X_train)

# preds_v1 = [pred[0] for pred in preds_v1]
print("Test R2: ",r2_score(test_v1["Fuel flow"] , preds))
print("Train R2: ",r2_score(train_v1["Fuel flow"],preds_train))
print("Test MAPE: ",mape(test_v1["Fuel flow"] , preds))
print("Train MAPE: ",mape(train_v1["Fuel flow"],preds_train))

from sklearn.tree import plot_tree

plt.figure(figsize=(25,10))
a = plot_tree(dtree, 
              feature_names=X_train.columns, 
              filled=True, 
              rounded=True, 
              fontsize=12)

# dtree=DecisionTreeRegressor(max_depth=4, random_state=111)
# dtree.fit(X_train,y_train)

Test R2: -5.3213071868417074
Train R2: 0.6853607387119951
Test MAPE: 0.0647192301824349
Train MAPE: 0.015408587505896926
<Figure size 1800x720 with 1 Axes>

In [0]:
plt.savefig('/dbfs/mnt/datalake/exploratory/Furnace_Analytics/60TPD/20201112_dcs_60_data_20200917_to_20201112.png')

<Figure size 432x288 with 0 Axes>

In [0]:
def reversal_outlier_treatment(df, agg_level):
    """
    This function cleans the DCS data based on reversal cycle
    
    Parameters
    ----------------------
    df: pd.DataFrame
    
    Return
    ---------------------
    input_df - pd.DataFrame
    """
    df.dropna(subset = ['Fuel flow'], inplace = True)
    for date in df['Date'].unique():

        df.loc[df['Date'] == date,"min"] = pd.cut(df.loc[df['Date'] == date,"Fuel flow"],10).value_counts().index[0].left
        df.loc[df['Date'] == date,"max"] = pd.cut(df.loc[df['Date'] == date,"Fuel flow"],10).value_counts().index[0].right
    df_v2 = df.loc[((df['Fuel flow'] >= df['min']) & (df['Fuel flow'] <= df['max']))]
    print ("\nOutliers (%age data points) removed from reversal cycle is ", np.round(1 - df_v2.shape[0]/df.shape[0] , 2))
    
    hourly_data = df_v2.resample(str(agg_level) + 'T').mean()

    return hourly_data
  
def load_dcs_data (required_columns = ['MCT','AirFlow','Fuel flow','ATF'], 
                   rename_dict= {'ThermocoupleMelterCrown3TCMC3PVDB249DD202':'MCT',
                            'SecondaryAirFlowvalueFTSAFPVDB205DD92':'AirFlow',
                            'MelterGasflowvalueFTMGFPVDB202DD92':'Fuel flow'}, agg_level = 10):
    """
    This function imports dcs data from DE pipeline
    
    Parameters
    ----------------------
    input_columns: list
    required_columns: list
    rename_dict: list
    
    Return
    ---------------------
    pd.DataFrame
    """
    dcs_df = pd.read_csv('/dbfs/mnt/datalake/master/dcs/master_dcs_60_tpd_live.csv')
    dcs_df['DateTime'] = pd.to_datetime(dcs_df['DateTime']) + timedelta(hours = 5, minutes = 30)
    dcs_df.rename(columns = rename_dict,inplace = True)
    dcs_df.set_index('DateTime', inplace = True)
    dcs_df['Date'] = dcs_df.index.date
    dcs_df.sort_index(inplace = True)
    hourly_df = reversal_outlier_treatment(dcs_df, agg_level)
    hourly_df['ATF'] = hourly_df['AirFlow']/hourly_df['Fuel flow']
    return hourly_df[required_columns]

In [0]:
def load_uems_data(rename_columns = {'STREAM1_INFERIOR_CV_NCV_Meter':'GAIL Ncv',
                          'STREAM2_INFERIOR_CV_NCV_Meter':'GSP Ncv'}):
    """
    This function imports UEMS data from DE pipeline and returns GAIL NCV
    
    Parameters
    ----------------------
    input_df: pd.DataFrame
    
    Return
    ---------------------
    input_df - pd.DataFrame
    """
    ncv = pd.read_csv("/dbfs/mnt/datalake/master/uem/ksb/15_min_tag_live.csv")
    ncv.sort_values(by = "From", inplace = True)
    ncv.rename(columns = rename_columns, inplace = True)

    return ncv

In [0]:
def load_app_data(furnace = "60"):
    """
    This function loads the app data recording MOT and draw
    
    Parameters
    ----------------------
    input_df: pd.DataFrame
    furnace = 95 TPD
    
    Return
    ---------------------
    input_df - pd.DataFrame
    """
    
    jdbcHostname = "pglpowerapps.database.windows.net"
    jdbcPort = "1433"
    jdbcDatabase = "PGL_Secondary_Resource" 
    user = "piramal"
    password = "admin@123" 

    table="dbo.data_furnace"

    jdbcDF = spark.read.format("jdbc").option("url", "jdbc:sqlserver://{0}:{1};database={2};".format(jdbcHostname,jdbcPort,jdbcDatabase)).option("dbtable", table).option("user", user).option("password",password).option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver").load()

    app_data = jdbcDF.toPandas()
    app_data['date'] = pd.to_datetime(app_data['Time_stamp']).dt.date
    app_data['hour'] = pd.to_datetime(app_data['Time_stamp']).dt.hour
    app_data['end hour'] = app_data['hour'] + 1
    app_data['parameter_value'] = pd.to_numeric(app_data['parameter_value'])
    filtered_data = app_data.loc[((app_data['Furance'] == furnace))]
    app_data_v2 = pd.pivot_table(filtered_data, index = ['date','hour','end hour'], columns = 'parameter', values = 'parameter_value', aggfunc='first').reset_index()
    app_data_v2.fillna(method = 'ffill', inplace = True)

    return app_data_v2

In [0]:
def dtree_prediction(model_df):
    """
    This function predicts fuel flow based on decision tree which is build on Airflow, MCT and NCV values
    
    Parameters
    ----------------------
    model_df: pd.DataFrame
    
    Return
    ---------------------
    model_df - pd.DataFrame
    """
    model_df['dtree_prediction'] = np.where(model_df['ATF']<=9.425,
                                                 np.where(model_df['Draw']<=50.625,417.122,
                                                         np.where(model_df['ATF']<=9.039,437.138,427.699)),
                                                         np.where(model_df['Draw']<=50.625,398.202,
                                                                  np.where(model_df['Draw']<=51.886,408.548,415.375)))
    
    return model_df


In [0]:
def create_model_df(hourly_df, ncv, app_data, required_columns = ['target-MCT','dtree_prediction','fuel_flow','ncv_values','ATF'], target_temp = 1553):
    """
    This function create model dataframe based on dcs and UEMS ncv dataframe
    
    Parameters
    ----------------------
    hourly_df: pd.DataFrame
    ncv: pd.DataFrame
    required_columns: list
    
    Return
    ---------------------
    model_df - pd.DataFrame
    """
    model_df = pd.DataFrame()
    model_df['fuel_flow'] = pd.Series(hourly_df['Fuel flow'].values[-1])
    model_df['ncv_values'] = pd.Series(ncv['GAIL Ncv'].values[-1])
    model_df['Draw'] = pd.Series(app_data['Furnace Draw'].values[-1])
    model_df['Current MCT'] = pd.Series(hourly_df['MCT'].values[-1])
    model_df['target-MCT'] = target_temp - model_df['Current MCT']
    model_df['ATF'] = pd.Series(hourly_df['ATF'].values[-1])
    model_df.index = pd.Series(hourly_df.index.values[-1])

    model_df = dtree_prediction(model_df)
    return model_df[required_columns]

In [0]:
def check(model_df, model_coefficient, cap = 10.03123):
    """
    This function applies the checks on the prediction and bound them to the cap mentioned
    
    Parameters
    ----------------------
    model_df: pd.DataFrame
    model_coefficient: pd.DataFrame
    cap: float
    
    Return
    ---------------------
    model_df - pd.DataFrame
    """
    model_df['target is close to target']  = np.where(np.abs(model_df['model_prediction'] - model_df['fuel_flow']).values[0] <=2.5,model_df['model_prediction'],model_df['fuel_flow'] - 0.6232131131)

    model_df['target > MOT'] = np.where(((model_df['model_prediction'] > model_df['fuel_flow'] - -0.5231) & (model_df['model_prediction'] < model_df['fuel_flow'] + cap)),model_df['model_prediction'], model_df['fuel_flow'] + min(cap,(model_df['target-MCT']*model_coefficient.iloc[1,1]).values[0]))

    model_df['target < MOT'] = np.where(((model_df['model_prediction'] > model_df['fuel_flow'] - cap) & (model_df['model_prediction'] < model_df['fuel_flow'] + 0.7231)),model_df['model_prediction'], model_df['fuel_flow'] - min(cap,(model_df['target-MCT']*model_coefficient.iloc[1,1]).values[0]))

    model_df['final_prediction'] = np.where(model_df['target-MCT'].between(0,1),model_df['target is close to target'], 
            np.where(model_df['target-MCT']>=1,model_df['target > MOT'],model_df['target < MOT']))

    return model_df

In [0]:
def model_prediction(model_df):
    """
    This function predicts the fuel flow based on linear regression model
    
    Parameters
    ----------------------
    model_df: pd.DataFrame
    
    Return
    ---------------------
    model_df - pd.DataFrame
    """
    model_coefficient = pd.read_excel('/dbfs/mnt/datalake/exploratory/Furnace_Analytics/Model_coefficients.xlsx', sheet_name = '60TPD')
    # model_coefficient
    if np.abs(model_df['target-MCT'].values[0]) > 2:
        model_coefficient.iloc[1,1] = model_coefficient.iloc[1,1]*2
    model_df['model_prediction'] = np.dot(model_coefficient.iloc[1:,1],model_df.iloc[0,:]) + model_coefficient.iloc[0,1]

    model_df = check(model_df, model_coefficient)

    model_df['Optimal ATF'] = np.where(model_df['ncv_values']<9800,'9.4-9.5','9.8-9.9')
    return model_df

In [0]:
hourly_df = load_dcs_data()
ncv = load_uems_data()
app_data = load_app_data(furnace = "60")
model_df = model_prediction(create_model_df(hourly_df, ncv,app_data))
model_df

,target-MCT,dtree_prediction,fuel_flow,ncv_values,ATF,model_prediction,target is close to target,target > MOT,target < MOT,final_prediction,Optimal ATF
2020-10-15 20:00:00,1.187744,415.375,441.992462,9845.27,9.707086,424.487063,441.369249,443.300443,440.684481,443.300443,9.8-9.9


In [0]:
hourly_df.tail(1)

,MCT,AirFlow,Fuel flow,ATF
DateTime,,,,
2020-10-15 19:30:00,1551.612256,4288.683887,443.829614,9.662906
